# End-to-end workflow in the H2O AI Cloud

Get started building and deploying machine learning models in the H2O AI Cloud.

* Create an AI engine for building models
* Use AutoML to build a machine learning model
* Deploy the model to production

## Create an AI engine
Create a Driverless AI engine for access to automated machine learning to build models for us on our data. 

See the `2 Managing AI Engines` tutorial for more details on how to use and interact with **Engine Manager** for creating and managing your AI Engines.

In [1]:
import os

import h2o_authn
import h2o_discovery
import h2o_engine_manager

In [ ]:
discovery = h2o_discovery.discover()

token_provider = h2o_authn.TokenProvider(
    refresh_token=os.getenv("H2O_CLOUD_CLIENT_PLATFORM_TOKEN"),
    issuer_url=discovery.environment.issuer_url,
    client_id=discovery.clients["platform"].oauth2_client_id,
)

engine_manager = h2o_engine_manager.login(token_provider=token_provider)

Creating a new engine can take 2-20 minutes depending on your environment and configuration settings. Please reach out to your admin as needed to reduce start times. 

In [ ]:
dai_engine = engine_manager.dai_engine_client.create_engine(
    display_name="My test engine",
)

dai_engine.wait()

In [ ]:
dai = dai_engine.connect()

## Build a model

### Import data

In [5]:
telco_churn = dai.datasets.create(
    data="https://h2o-internal-release.s3-us-west-2.amazonaws.com/data/Splunk/churn.csv",  
    data_source="s3", 
    name="Telco_Churn",
    force=True
)

Complete 100.00% - [4/4] Computed stats for column Churn?


In [6]:
print(telco_churn.key, "|", telco_churn.name)
print("\nColumns:", telco_churn.columns)
print('\nShape:', telco_churn.shape)

b34aade4-fcc5-11ec-bbbf-8af6970414ab | Telco_Churn

Columns: ['State', 'Account Length', 'Area Code', 'Phone', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?']

Shape: (3333, 21)


### Run an AutoML experiment

See the `3 AutoML` tutorial for more details on how to use and interact with Driverless AI for AutoML. 

In [ ]:
default_baseline = dai.experiments.create(
    name='Default Baseline', 
    train_dataset=telco_churn, 
    target_column="Churn?", 
    task="classification",
    accuracy=1, time=1, interpretability=6  # a quick AutoML experiment to see a baseline    
)

In [8]:
default_baseline.summary()

Status: Complete
Experiment: Default Baseline (bc7539d4-fcc5-11ec-bbbf-8af6970414ab)
  Version: 1.10.3, 2022-07-06 00:53
  Settings: 1/1/6, seed=6190721, GPUs enabled
  Train data: Telco_Churn (3333, 21)
  Validation data: N/A
  Test data: N/A
  Target column: Churn? (binary, 14.491% target class)
System specs: Docker/Linux, 54 GB, 32 CPU cores, 1/1 GPU
  Max memory usage: 0.763 GB, 0 GB GPU
Recipe: AutoDL (5 iterations, 2 individuals)
  Validation scheme: stratified, 1 internal holdout
  Feature engineering: 28 features scored (19 selected)
Timing: MOJO latency 0.0399 millis (214.2kB), Python latency 84.1064 millis (141.1kB)
  Data preparation: 15.50 secs
  Shift/Leakage detection: 1.18 secs
  Model and feature tuning: 17.01 secs (6 models trained)
  Feature evolution: 1.30 secs (0 of 3 model trained)
  Final pipeline training: 26.99 secs (7 models trained)
  Python / MOJO scorer building: 36.72 secs / 15.54 secs
Validation score: LOGLOSS = 0.4137853 (constant preds of -1.775)
Validat

## Deploy the model

### Create a project
We willl create a project for our use case to easily share our work with others and deploy the models

In [9]:
churn_project = dai.projects.create(
    name="Telco Churn Predictions", 
    description="Which of our customers is likely to cancel their contract?",
    force="True"
)

In [10]:
churn_project.link_experiment(default_baseline)

### Connect to MLOps
See the `5 Model Deployment` tutorial for more details on how to use and interact with MLOps for managing models and deploymnets. 

In [11]:
import h2o_mlops

In [ ]:
mlops = h2o_mlops.Client(
    gateway_url=discovery.services['mlops-api'].uri,
    token_provider=token_provider
)

### Create a deployment

In [13]:
my_project_deployment_environments = mlops.storage.deployment_environment.list_deployment_environments(body={
    'project_id': churn_project.key, 
}).deployment_environment

for de in my_project_deployment_environments:
    print(de.id, de.display_name)
    if de.display_name == "DEV":
        deployment_env = de.id

5ba7c218-8065-4e11-a819-c3bcbb991e1f DEV
9ad0c266-5913-4616-9057-9daa3188252f PROD


In [14]:
churn_model_deployed = mlops.storage.deployment_environment.deploy({
    'deployment_environment_id': de.id,
    'experiment_id': default_baseline.key,
    'type': 'SINGLE_MODEL'
})

### View information about our deployment

In [15]:
mlops.deployer.deployment_status.get_deployment_status({
    'deployment_id': churn_model_deployed.deployment.id
}).deployment_status

{'deployment_id': '',
 'grafana_endpoint': '',
 'last_modified_time': None,
 'message': '',
 'scorer': None,
 'state': 'DEPLOYMENT_STATE_UNSPECIFIED'}

### Wait for deployment to become healthy

In [16]:
import time

deployment_status = mlops.deployer.deployment_status.get_deployment_status({
        'deployment_id': churn_model_deployed.deployment.id
    }).deployment_status.state

while deployment_status != "HEALTHY":
    time.sleep(5)
    
    deployment_status = mlops.deployer.deployment_status.get_deployment_status({
        'deployment_id': churn_model_deployed.deployment.id
    }).deployment_status.state
    
mlops.deployer.deployment_status.get_deployment_status({
    'deployment_id': churn_model_deployed.deployment.id
}).deployment_status

{'deployment_id': '7b695155-de7e-4b27-b219-48d150ab6930',
 'grafana_endpoint': 'https://mlops-monitoring.cloud.h2o.ai/d/7b695155-de7e-4b27-b219-48d150ab6930',
 'last_modified_time': datetime.datetime(2022, 7, 6, 0, 57, 29, tzinfo=tzutc()),
 'message': 'N/A',
 'scorer': {'capabilities': {'method': 'GET',
                             'url': 'https://model.cloud.h2o.ai/7b695155-de7e-4b27-b219-48d150ab6930/model/capabilities'},
            'sample_request': {'method': 'GET',
                               'url': 'https://model.cloud.h2o.ai/7b695155-de7e-4b27-b219-48d150ab6930/model/sample_request'},
            'score': {'method': 'POST',
                      'url': 'https://model.cloud.h2o.ai/7b695155-de7e-4b27-b219-48d150ab6930/model/score'},
            'type': 'MOJO2_REST_SCORER'},
 'state': 'HEALTHY'}

In [17]:
sample_request = mlops.deployer.deployment_status.get_deployment_status({
    'deployment_id': churn_model_deployed.deployment.id
}).deployment_status.scorer.sample_request.url


deployment_url = mlops.deployer.deployment_status.get_deployment_status({
    'deployment_id': churn_model_deployed.deployment.id
}).deployment_status.scorer.score.url

### Make a prediction with new data

In [18]:
import requests
import json

In [19]:
sample_request_as_text = requests.get(sample_request).text
sample_request = json.loads(sample_request_as_text)
sample_request

{'fields': ['State',
  'Account Length',
  'Area Code',
  "Int'l Plan",
  'VMail Plan',
  'VMail Message',
  'Day Mins',
  'Day Calls',
  'Day Charge',
  'Eve Mins',
  'Eve Calls',
  'Eve Charge',
  'Night Mins',
  'Night Calls',
  'Night Charge',
  'Intl Mins',
  'Intl Calls',
  'Intl Charge',
  'CustServ Calls'],
 'rows': [['text',
   '0',
   '0',
   'text',
   'text',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0']]}

In [20]:
new_predictions = requests.post(
    url=deployment_url,
    json={
        'fields': sample_request["fields"],
        'rows': sample_request["rows"]
    }
)

predictions = json.loads(new_predictions.text)
predictions

{'fields': ['Churn?.False.', 'Churn?.True.'],
 'id': 'bc7539d4-fcc5-11ec-bbbf-8af6970414ab',
 'score': [['0.96486217', '0.035137847']]}

In [21]:
print(f"Churn Percent: {round(float(predictions['score'][0][1]) * 100, 1)}%")

Churn Percent: 3.5%


## Clean up
Delete our AutoML machine - we can import the project to any new Driverless AI machine, and the model will remain deployed to MLOps

In [22]:
dai_engine.delete()